# LangGraph Chatbot POC with CFG GenAI - Updated Version

This notebook demonstrates a comprehensive chatbot using LangGraph that:
1. Takes user queries
2. Determines intent and relevance
3. Converts natural language to SQL
4. Executes SQL queries against Snowflake
5. Formats responses back to natural language
6. Handles follow-up questions with context
7. **NEW**: Returns raw dataframe when formatting fails

## Features:
- **CFG GenAI Integration**: Uses proprietary LLM instead of OpenAI
- **Multi-table Support**: Handles complex queries across related tables
- **Context-Aware**: Processes follow-up questions intelligently
- **Real Database**: Direct Snowflake connection
- **Session Management**: Tracks conversation history
- **Robust Error Handling**: Returns structured data when LLM formatting fails
- **Table Formatting**: Automatic table formatting for query results

## 1. Setup and Dependencies

In [ ]:
# Install required packages (run this in your environment)
# !pip install langgraph pandas python-dotenv snowflake-connector-python cfggenaisdk

from snowflake.connector import DictCursor
import snowflake.connector
from cfggenaisdk import CFGGenAIGDK
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Any, TypedDict, Optional
from dataclasses import dataclass

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

print("✅ Dependencies loaded successfully!")

## 2. CFG GenAI Integration Setup

In [ ]:
# Initialize CFG GenAI GDK client
USECASE_ID = os.getenv("CFG_USECASE_ID", "chatbot_usecase")
EXPERIMENT_NAME = os.getenv("CFG_EXPERIMENT_NAME", "langgraph_chatbot")
EXPERIMENT_DESC = os.getenv("CFG_EXPERIMENT_DESC", "LangGraph chatbot with CFG GenAI")

try:
    gdk = CFGGenAIGDK(USECASE_ID, EXPERIMENT_NAME, EXPERIMENT_DESC)
    print("✅ CFG GenAI GDK initialized successfully!")
    print(f"Usecase ID: {USECASE_ID}")
    print(f"Experiment: {EXPERIMENT_NAME}")
except Exception as e:
    print(f"❌ Failed to initialize CFG GenAI GDK: {e}")
    raise e


def call_llm(prompt: str, system_prompt: str = "", model: str = None) -> str:
    """
    Robust helper function to make LLM calls using CFG GenAI GDK
    Handles different response structures
    """
    try:
        # Use model from environment variable if not specified
        if model is None:
            model = os.getenv("CFG_MODEL_ID", "md005_openai_gpt4o")
        
        # Combine system prompt and user prompt
        if system_prompt:
            combined_prompt = f"{system_prompt}\n\nUser Query: {prompt}"
        else:
            combined_prompt = prompt
        
        # Prepare prompt template for CFG GenAI
        prompt_template = {
            "prompt_template": [
                {"role": "system", "content": combined_prompt}
            ]
        }
        
        # Set hyperparameters
        hyperparam = {
            'max_tokens': 2000,
            'temperature': 0.1
        }
        
        # Make the LLM call
        gdk_response = gdk.invoke_llmgateway(
            prompt_template=prompt_template,
            hyperparam=hyperparam,
            model_id=model
        )
        
        # Try different ways to extract the content based on response structure
        generated_response = None
        
        # Method 1: Standard OpenAI-like structure
        try:
            if isinstance(gdk_response, dict):
                generated_response = gdk_response['genResponse']['choices'][0]['message']['content']
        except (KeyError, TypeError, IndexError):
            pass
        
        # Method 2: Direct choices access
        if generated_response is None:
            try:
                if isinstance(gdk_response, dict) and 'choices' in gdk_response:
                    generated_response = gdk_response['choices'][0]['message']['content']
            except (KeyError, TypeError, IndexError):
                pass
        
        # Method 3: Direct response field
        if generated_response is None:
            try:
                if isinstance(gdk_response, dict):
                    if 'response' in gdk_response:
                        generated_response = gdk_response['response']
                    elif 'content' in gdk_response:
                        generated_response = gdk_response['content']
                    elif 'text' in gdk_response:
                        generated_response = gdk_response['text']
            except (KeyError, TypeError):
                pass
        
        # Method 4: Tuple/List response
        if generated_response is None:
            try:
                if isinstance(gdk_response, (list, tuple)) and len(gdk_response) > 0:
                    first_element = gdk_response[0]
                    if isinstance(first_element, str):
                        generated_response = first_element
                    elif isinstance(first_element, dict):
                        if 'content' in first_element:
                            generated_response = first_element['content']
                        elif 'message' in first_element:
                            generated_response = first_element['message']
                        elif 'text' in first_element:
                            generated_response = first_element['text']
            except (IndexError, TypeError):
                pass
        
        # Method 5: String response
        if generated_response is None:
            try:
                if isinstance(gdk_response, str):
                    generated_response = gdk_response
            except Exception:
                pass
        
        # If all methods failed, return the raw response as string
        if generated_response is None:
            generated_response = str(gdk_response)
        
        return generated_response.strip() if generated_response else "No response generated"
        
    except Exception as e:
        print(f"❌ CFG GenAI LLM call failed: {e}")
        if 'gdk_response' in locals():
            print(f"Response type: {type(gdk_response)}")
            print(f"Response: {gdk_response}")
        return f"Error: {str(e)}"


print("✅ CFG GenAI integration setup complete!")

## 3. Test CFG GenAI Integration

In [ ]:
# Test the CFG GenAI integration
test_response = call_llm(
    prompt="What is 2 + 2?",
    system_prompt="You are a helpful assistant. Provide brief, accurate answers."
)

print(f"🧪 Test Response: {test_response}")
print("✅ CFG GenAI integration test successful!")

## 4. Database Schema and Snowflake Configuration

In [ ]:
class DatabaseSchema:
    """
    Load and manage database schema from external text file
    """

    def __init__(self, schema_file_path: str = "database_schema.txt"):
        self.schema_file_path = schema_file_path
        self.tables = {}
        self.relationships = {}
        self.raw_schema_text = ""
        self.load_schema()

    def load_schema(self):
        """
        Load database schema from text file - REQUIRED for operation
        """
        try:
            if os.path.exists(self.schema_file_path):
                with open(self.schema_file_path, 'r', encoding='utf-8') as file:
                    self.raw_schema_text = file.read()
                print(f"✅ Database schema loaded from {self.schema_file_path}")
                self.parse_schema()
            else:
                raise FileNotFoundError(
                    f"Schema file {self.schema_file_path} not found. Please create this file with your database schema.")
        except Exception as e:
            print(f"❌ Error loading schema file: {e}")
            raise e

    def parse_schema(self):
        """
        Parse the schema text file and extract table information
        """
        try:
            lines = self.raw_schema_text.strip().split('\n')
            current_table = None

            for line in lines:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue

                if line.upper().startswith('TABLE:') or line.upper().startswith('TABLE '):
                    current_table = line.split(':', 1)[1].strip() if ':' in line else line.split(' ', 1)[1].strip()
                    current_table = current_table.upper()
                    self.tables[current_table] = {
                        'description': f"Table: {current_table}",
                        'columns': {}
                    }

                elif current_table and ('|' in line or '\t' in line or '  ' in line):
                    parts = []
                    if '|' in line:
                        parts = [part.strip() for part in line.split('|')]
                    elif '\t' in line:
                        parts = [part.strip() for part in line.split('\t') if part.strip()]
                    else:
                        parts = [part.strip() for part in line.split() if part.strip()]

                    if len(parts) >= 2:
                        col_name = parts[0].upper()
                        col_type = parts[1].upper()
                        col_description = parts[2] if len(parts) > 2 else f"{col_name} column"

                        self.tables[current_table]['columns'][col_name] = {
                            'type': col_type,
                            'description': col_description
                        }

            print(f"✅ Parsed {len(self.tables)} tables from schema file")

        except Exception as e:
            print(f"❌ Error parsing schema: {e}")
            raise e

    def get_schema_description(self) -> str:
        if self.raw_schema_text and len(self.tables) > 1:
            schema_text = "DATABASE SCHEMA:\n\n"
            schema_text += self.raw_schema_text
            schema_text += "\n\nAVAILABLE TABLES:\n"
            for table_name in self.tables.keys():
                schema_text += f"- {table_name}\n"
            return schema_text
        else:
            schema_text = "DATABASE SCHEMA:\n\n"
            for table_name, table_info in self.tables.items():
                schema_text += f"Table: {table_name}\n"
                schema_text += f"Description: {table_info['description']}\n"
                schema_text += "Columns:\n"
                for col_name, col_info in table_info['columns'].items():
                    schema_text += f"  - {col_name} ({col_info['type']}): {col_info['description']}\n"
                schema_text += "\n"
            return schema_text

    def get_table_names(self) -> List[str]:
        return list(self.tables.keys())


class SnowflakeConfig:
    """
    Snowflake connection configuration
    """

    def __init__(self):
        self.account = os.getenv("SNOWFLAKE_ACCOUNT")
        self.user = os.getenv("SNOWFLAKE_USER")
        self.password = os.getenv("SNOWFLAKE_PASSWORD")
        self.warehouse = os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH")
        self.database = os.getenv("SNOWFLAKE_DATABASE")
        self.schema = os.getenv("SNOWFLAKE_SCHEMA", "PUBLIC")
        self.role = os.getenv("SNOWFLAKE_ROLE", "ACCOUNTADMIN")

        required_fields = {
            'account': self.account,
            'user': self.user,
            'password': self.password,
            'database': self.database
        }

        missing_fields = [field for field, value in required_fields.items() if not value]
        if missing_fields:
            raise ValueError(
                f"Missing required Snowflake configuration: {', '.join(missing_fields)}. Please check your .env file.")


# Global config instances
snowflake_config = None
database_schema = None


def initialize_configs():
    global snowflake_config, database_schema
    if snowflake_config is None:
        snowflake_config = SnowflakeConfig()
    if database_schema is None:
        database_schema = DatabaseSchema()
    return snowflake_config, database_schema


def get_snowflake_connection():
    global snowflake_config
    if snowflake_config is None:
        snowflake_config = SnowflakeConfig()

    try:
        conn = snowflake.connector.connect(
            account=snowflake_config.account,
            user=snowflake_config.user,
            password=snowflake_config.password,
            warehouse=snowflake_config.warehouse,
            database=snowflake_config.database,
            schema=snowflake_config.schema,
            role=snowflake_config.role
        )
        print("✅ Snowflake connection established successfully!")
        return conn
    except Exception as e:
        print(f"❌ Failed to connect to Snowflake: {e}")
        raise e


def execute_snowflake_query(sql_query: str) -> Dict[str, Any]:
    """
    Execute a SQL query against Snowflake and return results
    """
    conn = None
    try:
        conn = get_snowflake_connection()
        cursor = conn.cursor(DictCursor)
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description] if cursor.description else []

        print(f"📊 Query executed successfully. Returned {len(results)} rows.")

        return {
            "success": True,
            "data": results,
            "columns": columns,
            "row_count": len(results)
        }

    except Exception as e:
        print(f"❌ Error executing Snowflake query: {e}")
        return {
            "success": False,
            "error": str(e),
            "data": [],
            "columns": [],
            "row_count": 0
        }
    finally:
        if conn:
            conn.close()


print("✅ Database configuration setup complete!")

## 5. Initialize Configurations

In [ ]:
# Initialize configurations on startup to validate setup
try:
    initialize_configs()
    print("✅ All configurations initialized successfully!")
except Exception as e:
    print(f"⚠️ Configuration initialization failed: {e}")
    print("Please check your .env file and database_schema.txt file before running queries.")

## 6. State Management and Session Handling

In [ ]:
@dataclass
class ConversationTurn:
    """
    Represents a single conversation turn (query + response)
    """
    query: str
    response: str
    sql_query: str
    query_result: Dict[str, Any]
    timestamp: datetime
    intent: str


class SessionManager:
    """
    Manages conversation sessions and context for follow-up questions
    """

    def __init__(self):
        self.sessions: Dict[str, List[ConversationTurn]] = {}
        self.max_history = 5  # Keep last 5 conversation turns

    def add_turn(self, session_id: str, turn: ConversationTurn):
        """Add a conversation turn to the session history"""
        if session_id not in self.sessions:
            self.sessions[session_id] = []

        self.sessions[session_id].append(turn)

        # Keep only the last max_history turns
        if len(self.sessions[session_id]) > self.max_history:
            self.sessions[session_id] = self.sessions[session_id][-self.max_history:]

    def get_conversation_context(self, session_id: str) -> str:
        """Get formatted conversation context for LLM"""
        if session_id not in self.sessions or not self.sessions[session_id]:
            return "No previous conversation history."

        context = "PREVIOUS CONVERSATION HISTORY:\n"
        for i, turn in enumerate(self.sessions[session_id], 1):
            context += f"\n{i}. User: {turn.query}"
            context += f"\n   Response: {turn.response}"
            if turn.sql_query:
                context += f"\n   SQL Used: {turn.sql_query}"

        context += "\n\nUse this context to understand follow-up questions and references like 'that', 'those', 'compare to previous', etc."
        return context

    def clear_session(self, session_id: str):
        """Clear session history"""
        if session_id in self.sessions:
            del self.sessions[session_id]


class WorkflowState(TypedDict):
    """
    Enhanced state structure for our LangGraph workflow with conversation context
    """
    session_id: str
    user_query: str
    original_query: str
    intent: str
    is_relevant: bool
    is_followup: bool
    sql_query: str
    query_result: Dict[str, Any]
    final_response: str
    error: str
    context: Dict[str, Any]
    conversation_context: str


# Global session manager
session_manager = SessionManager()


def initialize_state(session_id: str, user_query: str) -> WorkflowState:
    """
    Initialize a new workflow state with conversation context
    """
    conversation_context = session_manager.get_conversation_context(session_id)

    return WorkflowState(
        session_id=session_id,
        user_query=user_query,
        original_query=user_query,
        intent="",
        is_relevant=False,
        is_followup=False,
        sql_query="",
        query_result={},
        final_response="",
        error="",
        context={},
        conversation_context=conversation_context
    )


print("✅ State management and session handling setup complete!")

## 7. Tool Functions - Core Processing Components

In [ ]:
def context_enhancer_tool(state: WorkflowState) -> WorkflowState:
    """
    Tool 0: Enhance user query with conversation context for follow-up questions
    """
    print(f"🔗 Enhancing query with context: {state['user_query']}")

    # Check if there's previous conversation context
    if state['conversation_context'] == "No previous conversation history.":
        print("✅ No previous context - using original query")
        return state

    # Use LLM to determine if this is a follow-up and enhance the query
    system_prompt = f"""You are a context enhancement assistant for a business data analysis chatbot.

{state['conversation_context']}

Your job is to:
1. Determine if the current user query is a follow-up question that references previous conversation
2. If it's a follow-up, enhance the query with proper context to make it standalone
3. If it's not a follow-up, return the original query unchanged

Follow-up indicators include:
- References like "that", "those", "it", "them"
- Comparative phrases like "compared to that", "vs the previous", "how about"
- Continuation phrases like "what about", "and for", "also show me"
- Time references building on previous queries like "and last month", "for the same period"

IMPORTANT: 
- If it's a follow-up, rewrite the query to be completely standalone and clear
- If it's NOT a follow-up, return exactly: "NOT_FOLLOWUP: [original query]"
- If it IS a follow-up, return: "FOLLOWUP: [enhanced standalone query]"

Examples:
- "What about premium customers?" → "FOLLOWUP: What are the sales metrics for premium customers?"
- "Compare that to last month" → "FOLLOWUP: Compare the transaction volume from the previous query to last month's transaction volume"
- "Show me product sales" → "NOT_FOLLOWUP: Show me product sales"
"""

    try:
        llm_response = call_llm(
            prompt=f"Current user query: '{state['user_query']}'",
            system_prompt=system_prompt
        )

        print(f"🤖 Context Enhancement Response: {llm_response}")

        if llm_response.startswith("FOLLOWUP:"):
            # This is a follow-up question - use enhanced query
            enhanced_query = llm_response.replace("FOLLOWUP:", "").strip()
            state['user_query'] = enhanced_query
            state['is_followup'] = True
            print(f"✅ Enhanced follow-up query: {enhanced_query}")
        else:
            # Not a follow-up - keep original query
            state['is_followup'] = False
            print("✅ Not a follow-up question - using original query")

    except Exception as e:
        state['error'] = f"Error in context enhancement: {str(e)}"
        print(f"❌ Error in context enhancement: {e}")

    return state


def intent_classifier_tool(state: WorkflowState) -> WorkflowState:
    """
    Tool 1: Determine if the user query is relevant to our use case using LLM
    """
    print(f"🔍 Analyzing intent for: {state['user_query']}")

    system_prompt = """You are an intent classifier for a comprehensive business data analysis system.
    
    Your job is to determine if a user query is relevant to business data analysis using our available tables:
    - TRANSACTIONS: Transaction records with volumes, dates, customer and product IDs
    - CUSTOMERS: Customer information including segments, locations, registration dates
    - PRODUCTS: Product catalog with categories, prices, brands, suppliers
    
    Relevant queries include:
    - Transaction analysis (volumes, amounts, counts, trends)
    - Customer analysis (segments, behavior, demographics, registration patterns)
    - Product analysis (categories, performance, pricing, brand analysis)
    - Sales and revenue analysis across any dimension
    - Time-based queries (last Friday, yesterday, this week, monthly trends, etc.)
    - Comparative analysis (compare periods, segments, products, customers)
    - Cross-table analysis (customer transaction patterns, product performance by segment, etc.)
    - Data aggregation requests (total, sum, average, count, etc.)
    - Business intelligence queries combining multiple data sources
    
    Irrelevant queries include:
    - General conversation, greetings
    - Questions about weather, news, personal topics
    - Technical support unrelated to data
    - Requests for information outside of our business data domain
    
    Respond with ONLY one of these formats:
    RELEVANT: data_query
    IRRELEVANT: general_conversation"""

    try:
        llm_response = call_llm(
            prompt=f"Classify this user query: '{state['user_query']}'",
            system_prompt=system_prompt
        )

        print(f"🤖 LLM Intent Response: {llm_response}")

        if "RELEVANT" in llm_response.upper():
            state['intent'] = "data_query"
            state['is_relevant'] = True
            print("✅ Query is relevant - proceeding with data analysis")
        else:
            state['intent'] = "irrelevant"
            state['is_relevant'] = False
            state['final_response'] = "I'm sorry, but I can only help with transaction and data-related queries. Please ask about transaction volumes, sales data, or similar topics."
            print("❌ Query is not relevant to our use case")

    except Exception as e:
        state['error'] = f"Error in intent classification: {str(e)}"
        print(f"❌ Error in intent classification: {e}")

    return state


print("✅ Context and Intent tools defined!")

In [ ]:
def nl_to_sql_tool(state: WorkflowState) -> WorkflowState:
    """
    Tool 2: Convert natural language to SQL using LLM with multi-table support
    """
    if not state['is_relevant']:
        return state

    print(f"🔄 Converting to SQL: {state['user_query']}")

    # Get current date for context
    current_date = datetime.now().strftime('%Y-%m-%d')
    current_day = datetime.now().strftime('%A')

    # Initialize configs and get the comprehensive schema description
    global snowflake_config, database_schema
    if snowflake_config is None:
        snowflake_config = SnowflakeConfig()
    if database_schema is None:
        database_schema = DatabaseSchema()

    schema_description = database_schema.get_schema_description()

    system_prompt = f"""You are an expert SQL query generator for a Snowflake database with multiple related tables.

{schema_description}

Database Configuration:
- Database: {snowflake_config.database}
- Schema: {snowflake_config.schema}
- Current date: {current_date} ({current_day})

SNOWFLAKE SQL GENERATION RULES:

1. QUERY STRUCTURE:
   - Generate ONLY the SQL query, no explanations or markdown
   - Use fully qualified table names: {snowflake_config.database}.{snowflake_config.schema}.TABLE_NAME
   - Always use uppercase for SQL keywords and table/column names

2. MULTI-TABLE QUERIES:
   - Use JOINs when the query requires data from multiple tables
   - Common patterns:
     * Customer analysis: JOIN TRANSACTIONS with CUSTOMERS
     * Product analysis: JOIN TRANSACTIONS with PRODUCTS  
     * Complete analysis: JOIN all three tables
   - Use appropriate JOIN types (INNER JOIN for existing relationships)

3. DATE FUNCTIONS (Snowflake-specific):
   - Current date: CURRENT_DATE()
   - Yesterday: DATEADD('day', -1, CURRENT_DATE())
   - Last Friday: DATEADD('day', -1, DATE_TRUNC('week', CURRENT_DATE()) + 4)
   - Last week: DATE_TRUNC('week', DATEADD('week', -1, CURRENT_DATE()))
   - This month: DATE_TRUNC('month', CURRENT_DATE())
   - Last month: DATE_TRUNC('month', DATEADD('month', -1, CURRENT_DATE()))

4. AGGREGATIONS:
   - Use SUM() for volume/amount calculations
   - Use COUNT() for transaction counts
   - Use AVG() for averages
   - Use GROUP BY for breakdowns by categories, dates, etc.
"""

    try:
        llm_response = call_llm(
            prompt=f"Convert this natural language query to SQL: '{state['user_query']}'",
            system_prompt=system_prompt
        )

        # Clean up the response to extract just the SQL
        sql_query = llm_response.strip()

        # Remove any markdown formatting or extra text
        if "```sql" in sql_query:
            sql_query = sql_query.split("```sql")[1].split("```")[0].strip()
        elif "```" in sql_query:
            sql_query = sql_query.split("```")[1].strip()

        # Remove any trailing semicolon and clean up
        sql_query = sql_query.rstrip(';').strip()

        state['sql_query'] = sql_query
        print(f"📝 Generated SQL: {state['sql_query']}")

    except Exception as e:
        state['error'] = f"Error generating SQL: {str(e)}"
        print(f"❌ Error in SQL generation: {e}")

    return state


def sql_executor_tool(state: WorkflowState) -> WorkflowState:
    """
    Tool 3: Execute the generated SQL query against Snowflake database
    """
    if not state['is_relevant'] or state['error'] or not state['sql_query']:
        return state

    print(f"⚡ Executing SQL against Snowflake: {state['sql_query']}")

    try:
        # Execute the query against Snowflake
        query_result = execute_snowflake_query(state['sql_query'])

        if query_result['success']:
            # Process the results into a format suitable for response generation
            data = query_result['data']

            if not data:
                state['query_result'] = {'message': 'No data found for the specified criteria'}
            elif len(data) == 1:
                # Single row result
                state['query_result'] = dict(data[0])
            else:
                # Multiple rows - convert to a more readable format
                if len(data) <= 10:  # For small result sets, include all data
                    state['query_result'] = {
                        'results': [dict(row) for row in data],
                        'row_count': len(data)
                    }
                else:
                    # For large result sets, provide summary
                    state['query_result'] = {
                        'sample_results': [dict(row) for row in data[:5]],
                        'total_rows': len(data),
                        'message': f'Showing first 5 of {len(data)} results'
                    }

            print(f"📊 Query executed successfully. Result: {state['query_result']}")

        else:
            # Query failed
            state['error'] = f"Snowflake query failed: {query_result['error']}"
            print(f"❌ Snowflake query failed: {query_result['error']}")

    except Exception as e:
        state['error'] = f"Error executing SQL: {str(e)}"
        print(f"❌ Error in SQL execution: {e}")

    return state


print("✅ SQL generation and execution tools defined!")

## 8. Enhanced Response Formatter with Table Support

In [ ]:
def format_results_as_table(results: List[Dict]) -> str:
    """
    Format query results as a simple table string
    """
    if not results:
        return "No data available"

    try:
        # Get all unique keys from all results
        all_keys = set()
        for result in results:
            all_keys.update(result.keys())

        headers = list(all_keys)

        # Create header row
        table_str = " | ".join(headers) + "\n"
        table_str += "-" * len(table_str) + "\n"

        # Add data rows
        for result in results:
            row_values = []
            for header in headers:
                value = result.get(header, "")
                # Format numbers with commas if they're large
                if isinstance(value, (int, float)) and abs(value) >= 1000:
                    value = f"{value:,}"
                row_values.append(str(value))
            table_str += " | ".join(row_values) + "\n"

        return table_str

    except Exception as e:
        # Fallback to simple string representation
        return "\n".join([str(result) for result in results])


def format_single_result(result: Dict) -> str:
    """
    Format a single result dictionary as a readable string
    """
    try:
        formatted_lines = []
        for key, value in result.items():
            # Format numbers with commas if they're large
            if isinstance(value, (int, float)) and abs(value) >= 1000:
                value = f"{value:,}"
            formatted_lines.append(f"{key}: {value}")

        return "\n".join(formatted_lines)

    except Exception as e:
        return str(result)


def response_formatter_tool(state: WorkflowState) -> WorkflowState:
    """
    Tool 4: Format the query results into natural language response using LLM
    If formatting fails, return the raw dataframe/results
    """
    if not state['is_relevant'] or state['error']:
        return state

    print("📝 Formatting response to natural language")

    system_prompt = """You are a data analyst assistant that converts query results into clear, natural language responses.

Guidelines:
1. Be conversational and helpful
2. Include specific numbers with proper formatting (commas for thousands)
3. For comparisons, calculate and mention percentage changes
4. Use clear date references
5. Keep responses concise but informative
6. If showing multiple data points, organize them clearly

Examples:
- Single value: "The total transaction volume for last Friday was 123,456."
- Comparison: "Transaction volume increased from 110,000 on August 22nd to 123,456 on August 29th, representing a 12.2% increase."
- Multiple values: "Here are the recent transaction volumes: August 29th: 123,456, August 28th: 98,765"
"""

    try:
        # Prepare the data context for the LLM
        data_context = f"""
Original Query: {state['user_query']}
SQL Query Used: {state['sql_query']}
Query Results: {json.dumps(state['query_result'], indent=2)}
"""

        llm_response = call_llm(
            prompt=f"Convert this query result into a natural language response:\n\n{data_context}",
            system_prompt=system_prompt
        )

        # Check if the LLM response is valid and not an error
        if llm_response and not llm_response.startswith("Error:") and len(llm_response.strip()) > 0:
            state['final_response'] = llm_response.strip()
            print(f"✅ Final response: {state['final_response']}")
        else:
            raise ValueError("LLM returned invalid or empty response")

    except Exception as e:
        print(f"⚠️ Error in response formatting: {e}")
        print("📊 Returning raw data instead of formatted response")

        # Return the raw dataframe/results when formatting fails
        query_result = state.get('query_result', {})

        if isinstance(query_result, dict):
            if 'results' in query_result:
                # Multiple rows - convert to DataFrame-like string representation
                results = query_result['results']
                if results:
                    # Create a simple table representation
                    df_str = format_results_as_table(results)
                    state['final_response'] = f"Here are the query results:\n\n{df_str}"
                else:
                    state['final_response'] = "No data found for your query."
            elif 'sample_results' in query_result:
                # Large result set - show sample
                sample_results = query_result['sample_results']
                total_rows = query_result.get('total_rows', len(sample_results))
                df_str = format_results_as_table(sample_results)
                state['final_response'] = f"Here are the first {len(sample_results)} results out of {total_rows} total:\n\n{df_str}"
            elif 'message' in query_result:
                # No data message
                state['final_response'] = query_result['message']
            else:
                # Single row or simple result
                df_str = format_single_result(query_result)
                state['final_response'] = f"Query result:\n\n{df_str}"
        else:
            # Fallback for unexpected result format
            state['final_response'] = f"Query completed. Result: {str(query_result)}"

        print(f"📋 Raw data response: {state['final_response'][:200]}...")

    return state


print("✅ Enhanced response formatter with table support defined!")

## 9. LangGraph Workflow Setup

In [ ]:
def create_workflow():
    """
    Create and configure the LangGraph workflow
    """
    # Create a new state graph
    workflow = StateGraph(WorkflowState)

    # Add nodes (our tool functions)
    workflow.add_node("context_enhancer", context_enhancer_tool)
    workflow.add_node("intent_classifier", intent_classifier_tool)
    workflow.add_node("nl_to_sql", nl_to_sql_tool)
    workflow.add_node("sql_executor", sql_executor_tool)
    workflow.add_node("response_formatter", response_formatter_tool)

    # Define the workflow edges (execution order)
    workflow.set_entry_point("context_enhancer")

    # Add conditional logic
    def after_context_enhancement(state: WorkflowState) -> str:
        """Move to intent classification after context enhancement"""
        if state['error']:
            return END
        else:
            return "intent_classifier"

    def should_continue(state: WorkflowState) -> str:
        """Decide whether to continue processing or end"""
        if not state['is_relevant']:
            return END
        elif state['error']:
            return END
        else:
            return "nl_to_sql"

    def after_sql_generation(state: WorkflowState) -> str:
        """Continue to SQL execution if no errors"""
        if state['error'] or not state['sql_query']:
            return END
        else:
            return "sql_executor"

    def after_sql_execution(state: WorkflowState) -> str:
        """Continue to response formatting if no errors"""
        if state['error']:
            return END
        else:
            return "response_formatter"

    # Add conditional edges
    workflow.add_conditional_edges(
        "context_enhancer",
        after_context_enhancement,
        {
            "intent_classifier": "intent_classifier",
            END: END
        }
    )

    workflow.add_conditional_edges(
        "intent_classifier",
        should_continue,
        {
            "nl_to_sql": "nl_to_sql",
            END: END
        }
    )

    workflow.add_conditional_edges(
        "nl_to_sql",
        after_sql_generation,
        {
            "sql_executor": "sql_executor",
            END: END
        }
    )

    workflow.add_conditional_edges(
        "sql_executor",
        after_sql_execution,
        {
            "response_formatter": "response_formatter",
            END: END
        }
    )

    # Response formatter always ends the workflow
    workflow.add_edge("response_formatter", END)

    # Compile the workflow
    app = workflow.compile()

    return app


# Create our workflow
chatbot_workflow = create_workflow()
print("✅ LangGraph workflow created successfully!")

## 10. Main Chatbot Function

In [ ]:
def process_query(user_query: str, session_id: str = "default") -> Dict[str, Any]:
    """
    Main function to process a user query through our LangGraph workflow

    Args:
        user_query: The user's natural language query
        session_id: Session identifier for context management

    Returns:
        Dictionary containing the final response and execution details
    """
    print(f"\n🚀 Processing query: '{user_query}'")
    print("=" * 50)

    # Initialize state
    initial_state = initialize_state(session_id, user_query)

    try:
        # Run the workflow
        final_state = chatbot_workflow.invoke(initial_state)

        # Prepare response
        response = {
            "query": user_query,
            "original_query": final_state.get('original_query', user_query),
            "enhanced_query": final_state.get('user_query', user_query),
            "is_followup": final_state.get('is_followup', False),
            "response": final_state.get('final_response', 'No response generated'),
            "session_id": session_id,
            "success": not bool(final_state.get('error')),
            "error": final_state.get('error', ''),
            "execution_details": {
                "intent": final_state.get('intent', ''),
                "is_relevant": final_state.get('is_relevant', False),
                "sql_query": final_state.get('sql_query', ''),
                "query_result": final_state.get('query_result', {})
            }
        }

        # Save conversation turn to session history (only if successful and relevant)
        if response['success'] and final_state.get('is_relevant', False):
            conversation_turn = ConversationTurn(
                query=final_state.get('original_query', user_query),
                response=final_state.get('final_response', ''),
                sql_query=final_state.get('sql_query', ''),
                query_result=final_state.get('query_result', {}),
                timestamp=datetime.now(),
                intent=final_state.get('intent', '')
            )
            session_manager.add_turn(session_id, conversation_turn)
            print(f"💾 Saved conversation turn to session {session_id}")

        print("=" * 50)
        print(f"✅ Final Response: {response['response']}")
        if response['is_followup']:
            print(f"🔗 Follow-up detected - Enhanced query: {response['enhanced_query']}")

        return response

    except Exception as e:
        error_response = {
            "query": user_query,
            "response": "I encountered an error while processing your request.",
            "session_id": session_id,
            "success": False,
            "error": str(e),
            "execution_details": {}
        }

        print(f"❌ Error processing query: {e}")
        return error_response


def clear_session(session_id: str):
    """
    Clear conversation history for a session
    """
    session_manager.clear_session(session_id)
    print(f"🗑️ Cleared session: {session_id}")


print("✅ Main chatbot function ready!")

## 11. Test the Enhanced Chatbot

In [ ]:
# Test queries - uncomment and run these to test your chatbot

# Test 1: Simple query
# result1 = process_query("What was the transaction volume last Friday?", "test_session")
# print(f"Response: {result1['response']}")

# Test 2: Follow-up question
# result2 = process_query("How does that compare to the previous week?", "test_session")
# print(f"Response: {result2['response']}")

# Test 3: Customer analysis (might trigger table formatting)
# result3 = process_query("Show me sales by customer segment this month", "test_session")
# print(f"Response: {result3['response']}")

# Test 4: Irrelevant query
# result4 = process_query("What's the weather like today?", "test_session")
# print(f"Response: {result4['response']}")

print("🧪 Test queries are ready to run!")
print("Uncomment the test queries above to try them out.")
print("\n🎯 Key Features:")
print("- ✅ CFG GenAI Integration")
print("- ✅ Context-aware follow-up questions")
print("- ✅ Multi-table SQL generation")
print("- ✅ Robust error handling")
print("- ✅ Automatic table formatting when LLM formatting fails")
print("- ✅ Session management")

## 12. Interactive Demo

In [ ]:
def interactive_demo():
    """
    Interactive demo function for testing the chatbot
    """
    print("🎉 Welcome to the Enhanced LangGraph Chatbot with CFG GenAI!")
    print("✨ NEW: Automatic table formatting when LLM formatting fails")
    print("Type 'quit' to exit, 'clear' to clear session history")
    print("=" * 60)
    
    session_id = "interactive_session"
    
    while True:
        try:
            user_input = input("\n💬 You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            elif user_input.lower() == 'clear':
                clear_session(session_id)
                continue
            elif not user_input:
                continue
            
            # Process the query
            result = process_query(user_input, session_id)
            
            # Display the response
            print(f"\n🤖 Bot: {result['response']}")
            
            # Show additional details if there was an error
            if not result['success'] and result['error']:
                print(f"❌ Error: {result['error']}")
                
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {e}")

# Uncomment the line below to start the interactive demo
# interactive_demo()

print("🎯 Interactive demo function ready!")
print("Call interactive_demo() to start chatting with the enhanced bot.")
print("\n📊 The bot will now automatically format results as tables when LLM formatting fails!")